# Target Dataframes: 
    
    

Concise: 
Unique ID | Listing Price | Asset creation date | Sale start | Sale duration | Primary Sale (boolean) | Number of Sales | Buy or Sale | Sale type | Creator Fee

Raw:
Everything

Responses:
Get responses

This notebook
1. Read orderbook_samples.csv, set listed_before variable to earliest timestamp. If no timestamp found, 
set listed_before to current time.
2. Call Orderbook API X number of times, get Y orders before listed_before.
3. Produce target dataframes. 
4. Append new data to orderbook_samples.csv

In [62]:
import pandas as pd
import requests
import os
from datetime import datetime



In [113]:
import time
time.time()

1635634166.1366937

In [115]:
datetime.timestamp(datetime.now())

1635634172.336621

In [125]:
def get_timestamp(file_name):
    #get unix timestamp for the created date in the last row of a file.
    # If file doesn't exit, gets current timestamp
    t0 = pd.Timestamp("1970-01-01")
    if os.path.exists(file_name):
        csv = pd.read_csv(file_name, index_col=False)
        data_time = pd.to_datetime(csv['created_date'])
        timestamps = data_time.apply(datetime.timestamp)

        return timestamps.values[-1]
    else:
        return datetime.timestamp(datetime.now())

In [136]:
def get_orders(listed_before, query_limit = 100, order_limit=50):
    #calls OpenSea orderbook API, creates a dataframe of length order_limit * query_limit
    df = pd.DataFrame()
    q = 0
    QUERY_COUNT = 0
    while True:
        url = f"https://api.opensea.io/wyvern/v1/orders?bundled=false&include_bundled=false&include_invalid=false&listed_before={listed_before}&limit={order_limit}&offset={order_limit*q}&order_by=created_date&order_direction=desc"
        headers = {"Accept": "application/json"}
        response = requests.request("GET", url, headers=headers)

        df = df.append(response_to_df(response),
                       ignore_index=True)
        df.sort_values('id',inplace=True)
        QUERY_COUNT += 1
        q += 1
        if QUERY_COUNT >= query_limit:
            print("Query count reached!")
            return df

In [68]:
# LIMIT = 10
# listed_before = 1635621155
# url = f"https://api.opensea.io/wyvern/v1/orders?bundled=false&include_bundled=false&include_invalid=false&listed_before={listed_before}&limit={LIMIT}&offset=0&order_by=created_date&order_direction=desc"
# headers = {"Accept": "application/json"}
# response = requests.request("GET", url, headers=headers)
# df = response_to_df(response)

In [69]:
def response_to_df(response):
    # Clean up the response.json() a bit, and extracts some information from dictionary columns
    df = pd.DataFrame(response.json()['orders'])
    asset_unique_id = pd.DataFrame(pd.DataFrame(
        df["metadata"].tolist())['asset'].tolist())
    df['asset_unique_id'] = asset_unique_id[['id', 'address']].agg('-'.join,
                                                               axis=1)
    df['number_sales'] = pd.DataFrame(df["asset"].tolist())['num_sales']
    df['creator_fee'] = pd.DataFrame(
        pd.DataFrame(df["asset"].tolist())
        ['asset_contract'].tolist())['seller_fee_basis_points']
    df['description'] = pd.DataFrame(df["asset"].tolist())['description']
    df['external_link'] = pd.DataFrame(df["asset"].tolist())['external_link']
    df['image_url'] = pd.DataFrame(df["asset"].tolist())['image_url']
    df['animation_url'] = pd.DataFrame(df["asset"].tolist())['animation_url']
    df.drop(columns="asset", inplace=True)
    return df

In [70]:
def save_df(df, file_name, append=True):
    # Save df to a .csv, supports appending
    if append and os.path.exists(file_name):
        df_read = pd.read_csv(file_name, index_col=False)
        df_read = df_read.append(df, ignore_index=True)
        #df_read.drop_duplicates(keep="first", inplace=True) # Doesn't work for some reason! also breaks with dict
        df_read.to_csv(file_name, index=False)
    else:
        df.to_csv(file_name, index=False)


In [139]:
file_name = 'orderbook_samples.csv'
listed_before_timestamp = get_timestamp(file_name)
df = get_orders(listed_before_timestamp,order_limit= 10,query_limit=5)
save_df(df,file_name)

Query count reached!
